In [ ]:
# encoder

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input

In [ ]:
# Load data
from tensorflow.keras.datasets import mnist
(trainSamples, trainLabels), (testSamples, testLabels) = mnist.load_data()

trainSamples = trainSamples.astype('float32') / 255.0
testSamples = testSamples.astype('float32') / 255.0
trainSamples = trainSamples.reshape(len(trainSamples),-1)
testSamples = testSamples.reshape(len(testSamples),-1)


noise_factor = 0.4
trainSamples_noisy = trainSamples + noise_factor * np.random.normal(size=trainSamples.shape) 
testSamples_noisy = testSamples + noise_factor * np.random.normal(size=testSamples.shape)

trainSamples_noisy = np.clip(trainSamples_noisy, 0.0, 1.0)
testSamples_noisy = np.clip(testSamples_noisy, 0.0, 1.0)

print(trainSamples.shape)
print(testSamples.shape)
print(trainLabels.shape)




In [ ]:
input_size = 28*28 #784
code_size = 32

input_img = Input(shape=(input_size,))
code = Dense(code_size, activation='relu')(input_img)
output_img = Dense(input_size, activation='sigmoid')(code)

# input_img = Input(shape=(input_size,))
# hidden_1 = Dense(128, activation='relu')(input_img)
# code = Dense(code_size, activation='relu')(hidden_1)
# hidden_2 = Dense(128, activation='relu')(code)
# output_img = Dense(input_size, activation='sigmoid')(hidden_2)


autoencoder = Model(input_img, output_img)

autoencoder.summary()
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.fit(trainSamples, trainSamples, epochs=5)
#autoencoder.fit(trainSamples_noisy, trainSamples, epochs=5)


In [ ]:
show_weights = True

resultSamples = autoencoder.predict(testSamples)
n = 5
plt.figure(figsize=(12, 12))

print('Original Images')
for i in range(n):
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(testSamples[i].reshape((28,28)))
    plt.gray()
    plt.axis("off")
plt.show()

if show_weights:
    print("Input layer")
    plt.figure(figsize=(12, 12))
    weights = autoencoder.get_weights()[0].T
    print("Weights:",weights.T.shape)
    size = weights.shape[0]
    for i in range(size):
            ax = plt.subplot(1, size, i + 1)
            plt.imshow(weights[i].reshape((28,28)))
            plt.gray()
            plt.axis("off")
    plt.show()

    print("Output layer")
    plt.figure(figsize=(12, 12))
    weights = autoencoder.get_weights()[2] ## [1]- biases!
    print("Weights:",weights.shape)
    size = weights.shape[0]
    for i in range(size):
            ax = plt.subplot(1, size, i + 1)
            plt.imshow(weights[i].reshape((28,28)))
            plt.gray()
            plt.axis("off")
    plt.show()

plt.figure(figsize=(12, 12))
print('Reconstructed Images')
for i in range(n):
        # plot reconstruction 
        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(resultSamples[i].reshape((28,28)))
        plt.gray()
        plt.axis("off")
plt.show()



In [ ]:
resultSamples = autoencoder.predict(testSamples_noisy)
n = 5
print('Original Images')
plt.figure(figsize=(12, 12))
for i in range(n):
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(testSamples[i].reshape((28,28)))
    plt.gray()
    plt.axis("off")
plt.show()

print('Noisy Images')
plt.figure(figsize=(12, 12))
for i in range(n):
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(testSamples_noisy[i].reshape((28,28)))
    plt.gray()
    plt.axis("off")
plt.show()

print('Reconstructed Images')
plt.figure(figsize=(12, 12))
for i in range(n):
        # plot reconstruction 
        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(resultSamples[i].reshape((28,28)))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
plt.show()



In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt


points = 20
encoder = Model(input_img, code)
vectors = encoder.predict(testSamples[:points])
pca = PCA(n_components=2)
vectors2D = pca.fit_transform(vectors)
plt.scatter(vectors2D[:,0],vectors2D[:,1], c=testLabels[:points], cmap=plt.get_cmap('plasma'))
for i,w in enumerate(vectors):
    plt.annotate(testLabels[i],(vectors2D[i,0],vectors2D[i,1]))
           
plt.show()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,cohen_kappa_score

encoder = Model(input_img, code)
testVectors = encoder.predict(testSamples)
trainVectors = encoder.predict(trainSamples)

# for orignal images (works very long - 784 dimansions!)
# testVectors = testSamples.reshape(testSamples.shape[0],-1)
# trainVectors = trainSamples.reshape(trainSamples.shape[0],-1)

knn_model = KNeighborsClassifier()
knn_model.fit(trainVectors, trainLabels)    
predLabels = knn_model.predict(testVectors)

print(confusion_matrix(testLabels, predLabels))
print(classification_report(testLabels, predLabels))

